In [4]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Load the data

In [6]:
data = pd.read_csv('../data/amazonreviews.tsv', sep='\t')
data.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


## Clean the data

In [8]:
# NaN values
data.isnull().sum()

label     0
review    0
dtype: int64

In [10]:
data = data.replace('', float('NaN'))

In [11]:
data.isnull().sum()

label     0
review    0
dtype: int64

It does not have and missing value or empty strings

In [14]:
data.label.value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

## Model

Now, let's import a model called VADER (Valence Aware Dictionary for Sentiment Reasoning), which is used for text sentiment analysis that is sensitive to both polarity (pos/neg) and intensity (strength) of emotion. VADER sentimental analysis relies on a dictionary that maps lexical features to emotion intensities known as sentiment scores. The sentiment score of a text can be obtained by summing up the intensity of each word in the text.

In [16]:
sid = SentimentIntensityAnalyzer() 

In [22]:
review = data.loc[0, 'review']
print(f"{review} \n\n{sid.polarity_scores(review)}")

Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^ 

{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'compound': 0.9454}


In [25]:
data['scores'] = data['review'].apply(lambda review: sid.polarity_scores(review))
data['comp_scores'] = data['scores'].apply(lambda score: score['compound'])
data['predicted'] = data['comp_scores'].apply(lambda comp_score: 'pos' if comp_score >= 0 else 'neg')
data.head()

,label,review,scores,comp_scores,predicted
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454,pos
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957,pos
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858,pos
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814,pos
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781,pos


## Performance

In [27]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [31]:
accuracy_score(data.label, data.predicted)

0.7097

In [32]:
print(classification_report(data.label, data.predicted))

              precision    recall  f1-score   support

         neg       0.86      0.52      0.64      5097
         pos       0.64      0.91      0.75      4903

    accuracy                           0.71     10000
   macro avg       0.75      0.71      0.70     10000
weighted avg       0.75      0.71      0.70     10000



In [33]:
print(confusion_matrix(data.label, data.predicted))

[[2629 2468]
 [ 435 4468]]


So, it looks like VADER couldn't judge the reviews very accurately. This demonstrates one of the biggest challenges in sentiment analysis - understanding human semantics. Many of the reviews had positive things to say about a movie, reserving final judgement to the last sentence.